In [2]:
!pip install selenium

In [47]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time


# 인기급상승 페이지 열기
driver = webdriver.Chrome()
driver.get('https://www.youtube.com/feed/trending')

# 인기급상승 동영상만 불러오기
main_youtube = driver.find_elements(By.ID, 'grid-container')

# 스크롤
action = driver.find_element(By.TAG_NAME, 'body')
action.send_keys(Keys.PAGE_DOWN)
action.send_keys(Keys.PAGE_DOWN)


img_urls = [] # 썸네일 이미지 다운로드
title_info = [] # 동영상 제목, 채널 이름, 조회수 등의 정보
video_urls = [] # 동영상 주소


for i in main_youtube :
    for j in i.find_elements(By.CLASS_NAME, 'yt-core-image.yt-core-image--fill-parent-height.yt-core-image--fill-parent-width.yt-core-image--content-mode-scale-aspect-fill.yt-core-image--loaded') :
        img_urls.append(j.get_attribute('src'))
    for t in i.find_elements(By.ID, 'video-title') :
        title_info.append(t.get_attribute('aria-label'))
        video_urls.append(t.get_attribute('href'))


# 동영상의 제목과 채널 이름만 저장
title_names = []
title_ch = []
for t in title_info :
    title_names.append(t.split(' 게시자: ')[0])
    title_ch.append(t.split(' 게시자: ')[1].split(' 조회수')[0])


# 썸네일 이미지를 채널 이름을 제목으로 하여 저장 (top5개만 하기 위하여)
import urllib.request

for i in range(5) : 
    urllib.request.urlretrieve(img_urls[i], "플젝이미지/{0}.jpg".format(title_ch[i]))


driver.close()

In [48]:
import pandas as pd 

data = {'title_name' : title_names[0:5] ,
        'title_ch' : title_ch[0:5] ,
        'video_urls' : video_urls[0:5]}

video_info = pd.DataFrame(data)

In [49]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time

video_reply = []

for url in video_info['video_urls'] :
    driver = webdriver.Chrome()
    driver.get(url)

    # 첫번째스크롤 : 댓글 로딩 , 두번째스크롤 : +20 댓글 로딩 , 세번째스크롤 + 20 댓글 로딩 -> 총 40개의 댓글 로딩

    for _ in range(3) : 
        action = driver.find_element(By.TAG_NAME, 'body')
        action.send_keys(Keys.END)
        time.sleep(2)

    # 로딩 시간을 위해
    time.sleep(2)

    # 댓글구역을 찾기 위해 xpath로 찾아감
    target_xpath = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-comments/ytd-item-section-renderer'
    element = driver.find_element(By.XPATH, target_xpath)

    reply = []
    for r in element.find_elements(By.CSS_SELECTOR, 
                                'span.yt-core-attributed-string.yt-core-attributed-string--white-space-pre-wrap') :
        reply.append(r.text)
    
    video_reply.append(reply)

    driver.close()


In [55]:
video_info['video_reply'] = video_reply

In [27]:
print(video_info)

                                          title_name        title_ch  \
0                               [한글자막] 아이브 장원영에게 예쁘다  피식대학Psick Univ   
1                              임영웅 '온기' Official M/V             임영웅   
2  원우 케리아가 진짜라고? 이게 된다..진짜란다😦ㅣ세븐틴 원우&T1 케리아 [셀폰KODE]         KODE 코드   
3         [여행 브이로그] 아기와 해외 여행을 간다면?👶🏻💟 | 조카와 첫 해외여행!            지유롭게   
4                                       콘치즈 하나 더 주세요             숏박스   

                                    video_urls  \
0  https://www.youtube.com/watch?v=tCCtZ6J5vs8   
1  https://www.youtube.com/watch?v=2WGc-5VdoGw   
2  https://www.youtube.com/watch?v=gJEb8oa5jXo   
3  https://www.youtube.com/watch?v=yotiO6FdlyY   
4  https://www.youtube.com/watch?v=n9yXBQIzO2w   

                                         video_reply  
0  [썸넬에 자기들 사진 아예 빼버리고 장원영 사진만 넣어놨어ㅋㅋㅋㅋㅋㅋㅋㅋㅋ최고의 선...  
1  [🩵 임영웅 신곡 "온기" \n남루한 모습에서 풍기는 처절함 \n거친 모습에서 보이...  
2  [Can't believe we're playing KODE superstar Ke...  
3  [지유님 매번 느끼긴했지만 새삼 말투에서도 오는 착함이 항상 뿜뿜해요,, 예쁜

In [56]:
video_info.to_csv('video_info.csv', encoding='utf-16')